In [2]:
!pip3 install ktrain

In [3]:
import os
import string
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import tensorflow as tf
import ktrain
from ktrain import text
from spacy import displacy
from sklearn.model_selection import train_test_split

nlp = spacy.load('en_core_web_lg')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
dir(text)

['AnswerExtractor',
 'EnglishTranslator',
 'SimpleQA',
 'TextExtractor',
 'Transformer',
 'TransformerEmbedding',
 'TransformerSummarizer',
 'Translator',
 'ZeroShotClassifier',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'data',
 'eda',
 'entities_from_array',
 'entities_from_conll2003',
 'entities_from_df',
 'entities_from_gmb',
 'entities_from_txt',
 'extract_filenames',
 'filter_by_id',
 'get_topic_model',
 'learner',
 'load_text_files',
 'load_topic_model',
 'models',
 'ner',
 'pickle',
 'predictor',
 'preprocessor',
 'print_sequence_taggers',
 'print_text_classifiers',
 'print_text_regression_models',
 'qa',
 'seqlen_stats',
 'sequence_tagger',
 'shallownlp',
 'summarization',
 'text_classifier',
 'text_regression_model',
 'textextractor',
 'texts_from_array',
 'texts_from_csv',
 'texts_from_df',
 'texts_from_folder',
 'textutils',
 'translation',
 'zsl']

## Read Data

In [5]:
root_folder = r'/kaggle/input/20news-19997/20_newsgroups'
sub_folders = os.listdir(root_folder)
len(sub_folders)

20

In [6]:
sub_folders

['soc.religion.christian',
 'talk.politics.misc',
 'rec.motorcycles',
 'talk.politics.mideast',
 'sci.electronics',
 'sci.space',
 'rec.sport.baseball',
 'misc.forsale',
 'talk.politics.guns',
 'sci.med',
 'comp.sys.mac.hardware',
 'rec.sport.hockey',
 'comp.sys.ibm.pc.hardware',
 'talk.religion.misc',
 'sci.crypt',
 'comp.windows.x',
 'comp.os.ms-windows.misc',
 'rec.autos',
 'comp.graphics',
 'alt.atheism']

In [7]:
def read_data(root_folder, sub_folders):

    data, labels = [], []
    
    for folder in sub_folders:
        path = os.path.join(root_folder, folder)
        samples = os.listdir(path)

        for sample in samples:
            sample_path = os.path.join(path, sample)
            text = open(sample_path, encoding='iso-8859-1')   # 'latin-1')
            data.append(text.read())
            labels.append(folder)
    
    return data, labels

In [8]:
data, labels = read_data(root_folder, sub_folders)

print(len(data))
print(len(labels))

19997
19997


In [9]:
text.texts_from_array?

Signature:
text.texts_from_array(
    x_train,
    y_train,
    x_test=None,
    y_test=None,
    class_names=[],
    max_features=20000,
    maxlen=400,
    val_pct=0.1,
    ngram_range=1,
    preprocess_mode='standard',
    lang=None,
    random_state=None,
    verbose=1,
)
Docstring:
```
Loads and preprocesses text data from arrays.
texts_from_array can handle data for both text classification
and text regression.  If class_names is empty, a regression task is assumed.
Args:
    x_train(list): list of training texts
    y_train(list): labels in one of the following forms:
                   1. list of integers representing classes (class_names is required)
                   2. list of strings representing classes (class_names is not needed and ignored.)
                   3. a one or multi hot encoded array representing classes (class_names is required)
                   4. numerical values for text regresssion (class_names should be left empty)
    x_test(list): list of training 

In [10]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=data,
                                                                      y_train=labels,
                                                                      val_pct=0.2,
                                                                      maxlen=512,
                                                                      random_state=None,
                                                                      preprocess_mode='bert')

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [11]:
x_train

[array([[  101,  4130,  1024, ..., 26775, 22571,   102],
        [  101,  4130,  1024, ...,  2015,  1012,   102],
        [  101,  2739, 17058, ...,     0,     0,     0],
        ...,
        [  101,  1060,  2890, ...,  1012,  1045,   102],
        [  101,  2739, 17058, ...,  1066,  1032,   102],
        [  101,  2739, 17058, ...,     0,     0,     0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [12]:
print(len(x_train))
print(len(x_train[0]))
print(len(x_train[1]))

2
15997
15997


In [13]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
print(len(y_train))
print(len(y_train[0]))

15997
20


In [15]:
preproc

In [16]:
dir(preproc)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_transform_y',
 'c',
 'check_trained',
 'get_classes',
 'get_preprocessor',
 'get_tokenizer',
 'is_chinese',
 'is_nospace_lang',
 'lang',
 'max_features',
 'maxlen',
 'migrate_classes',
 'multilabel',
 'ngram_range',
 'preprocess',
 'preprocess_test',
 'preprocess_train',
 'preprocess_train_called',
 'print_seqlen_stats',
 'process_chinese',
 'seqlen_stats',
 'set_classes',
 'set_multilabel',
 'tok',
 'tok_dct',
 'undo',
 'ytransform']

## Model

In [17]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 512


/opt/conda/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [18]:
learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

## Training

In [19]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
2667/2667 [==============================] - 1457s 538ms/step - loss: 0.5406 - accuracy: 0.8445 - val_loss: 0.0682 - val_accuracy: 0.9718


In [20]:
learner.evaluate((x_train, y_train))

500/500 [==============================] - 367s 725ms/step
              precision    recall  f1-score   support

           0       1.00      0.78      0.87       830
           1       0.98      0.99      0.99       790
           2       1.00      0.98      0.99       800
           3       0.99      0.98      0.99       806
           4       0.99      0.99      0.99       795
           5       1.00      0.99      0.99       788
           6       0.96      1.00      0.98       799
           7       1.00      0.98      0.99       787
           8       1.00      1.00      1.00       802
           9       1.00      1.00      1.00       813
          10       1.00      1.00      1.00       811
          11       1.00      1.00      1.00       799
          12       1.00      0.99      0.99       791
          13       0.99      1.00      1.00       805
          14       1.00      1.00      1.00       780
          15       1.00      0.99      1.00       795
          16       0.9

array([[645,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 185],
       [  0, 786,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          3,   1,   0,   0,   0,   0,   0],
       [  0,   4, 788,   5,   0,   1,   1,   0,   0,   0,   0,   0,   0,
          1,   0,   0,   0,   0,   0,   0],
       [  0,   2,   3, 792,   5,   0,   4,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 791,   0,   4,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   8,   0,   0,   0, 779,   1,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 799,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  10, 775,   0,   0,   0,   0,   2,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   3,   0, 799,   0,   0,  

In [21]:
learner.evaluate((x_test, y_test))

125/125 [==============================] - 91s 725ms/step
              precision    recall  f1-score   support

           0       1.00      0.81      0.89       170
           1       0.98      0.99      0.98       210
           2       0.99      0.99      0.99       200
           3       1.00      0.99      1.00       194
           4       1.00      1.00      1.00       205
           5       1.00      0.98      0.99       212
           6       0.97      1.00      0.99       201
           7       1.00      0.99      0.99       213
           8       1.00      0.99      0.99       198
           9       1.00      1.00      1.00       187
          10       1.00      1.00      1.00       189
          11       1.00      1.00      1.00       201
          12       0.99      0.99      0.99       209
          13       0.99      1.00      0.99       195
          14       1.00      1.00      1.00       220
          15       1.00      1.00      1.00       202
          16       0.93

array([[137,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,  33],
       [  0, 208,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          2,   0,   0,   0,   0,   0,   0],
       [  0,   1, 199,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   1,   0, 193,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0, 205,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   2,   1,   0,   0, 207,   1,   0,   0,   0,   0,   0,   1,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 201,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   2, 210,   0,   0,   0,   0,   1,
          0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   2,   0, 196,   0,   0,  

## deployment
https://medium.com/analytics-vidhya/how-to-deploy-your-neural-network-model-using-ktrain-ae255b134c77

## save 

In [34]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/kaggle/working/')

## Load

In [ ]:
import pickle
from tensorflow.keras.models import load_model

# loading preprocess and model file
features = pickle.load(open('/kaggle/working/tf_model.preproc','rb'))
new_model = load_model('/kaggle/working/tf_model.h5')

## download

In [42]:
!cd /kaggle/working/

In [43]:
!ls

__notebook_source__.ipynb  tf_model.h5	tf_model.preproc


In [38]:
from IPython.display import FileLink

FileLink(r'tf_model.h5')

/kaggle/working/tf_model.h5

In [40]:
FileLink(r'tf_model.preproc')

/kaggle/working/tf_model.preproc